# Textcuboid


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import re
import os
import string
import json
import keras
import wordninja
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
TRAIN_DATA_PATH = "C:/Users/user/Desktop/bilm-tf-master/textdataset/IMDB/aclImdb/train"
TEST_DATA_PATH = "C:/Users/user/Desktop/bilm-tf-master/textdataset/IMDB/aclImdb/test"

def read_text_file(path):
    labels = ['neg','pos']
    if os.path.exists(path):
        text=[]
        text_label =[]
        for directory_name in os.listdir(path):
            if directory_name in labels:
                label_index = labels.index(directory_name)
                data_path = os.path.join(path,directory_name)
                for file in os.listdir(data_path):
                    with open(os.path.join(data_path,file),'r', encoding='utf-8') as f:
                        text.append(f.read())
                        text_label.append(label_index)
        return pd.DataFrame(text,columns =['texts']),pd.DataFrame(text_label,columns =['label'])
    
x_train,y_train = read_text_file(TRAIN_DATA_PATH) 
x_test,y_test = read_text_file(TEST_DATA_PATH) 

train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_test, y_test], axis=1)

train.drop_duplicates(inplace=True)
test.drop_duplicates(inplace=True)

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

train[['texts']] = train[['texts']].applymap(lambda x:remove_punctuation(x))
train[['texts']] = train[['texts']].applymap(lambda x:x.lower())
test[['texts']] = test[['texts']].applymap(lambda x:remove_punctuation(x))
test[['texts']] = test[['texts']].applymap(lambda x:x.lower())

def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    #text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    return text

X_train = train.drop(columns=['label'])
X_test = test.drop(columns=['label'])
y_train = train['label']
y_test = test['label']

X_train = X_train['texts'].apply(clean_text)
X_test = X_test['texts'].apply(clean_text)

pattern = '[^a-z ]'
Clean_X_train=[]
Clean_X_test=[]

for sen in X_train:
    Clean_X_train.append(re.sub(pattern, '', str(sen)))
    
for sen in X_test:
    Clean_X_test.append(re.sub(pattern, '', str(sen)))

y_train=list(y_train)
y_test=list(y_test)

train_df = pd.DataFrame({'X_train': Clean_X_train, 'y_train': y_train})
test_df = pd.DataFrame({'X_test': Clean_X_test, 'y_test': y_test})

# 레이블 값에 따라 데이터프레임을 그룹화하고 각 그룹에서 8000개의 샘플을 랜덤하게 추출
train_df = train_df.groupby('y_train').apply(lambda x: x.sample(n=8000, random_state=42))

# 레이블 값에 따라 데이터프레임을 그룹화하고 각 그룹에서 2000개의 샘플을 랜덤하게 추출
test_df = test_df.groupby('y_test').apply(lambda x: x.sample(n=2000, random_state=42))

# 인덱스를 재설정합니다. drop=True 옵션을 사용하여 기존 인덱스를 제거합니다.
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

x_train = train_df['X_train'].tolist()
y_train = train_df['y_train'].tolist()
x_test = test_df['X_test'].tolist()
y_test = test_df['y_test'].tolist()

to_txt=x_train+x_test
y=y_train+y_test

In [3]:
encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=list(label[:16000])
y_test=list(label[16000:])

In [4]:
#수동으로 cross validation을 하기위한 코드
cnt=5  #cnt가 2일때 두번째 시행
cnt=cnt-2

def exclude_list(input_list, cnt):
    return input_list[:cnt*2000]+input_list[cnt*2000+2000:8000]+input_list[8000:cnt*2000+8000]+input_list[cnt*2000+10000:]

if cnt>-1:
    x_train_cnt=exclude_list(x_train,cnt)+x_test
    y_train_cnt=exclude_list(y_train,cnt)+y_test
    x_test_cnt=x_train[cnt*2000:cnt*2000+2000]+x_train[cnt*2000+8000:cnt*2000+10000]
    y_test_cnt=y_train[cnt*2000:cnt*2000+2000]+y_train[cnt*2000+8000:cnt*2000+10000]

In [5]:
#텍스트 큐보이드 생성을 위한 인덱스
def train_idx_list(cnt):
    original_train_lst=[i for i in range(16000)]
    return original_train_lst[:cnt*2000]+original_train_lst[cnt*2000+2000:8000]+original_train_lst[8000:cnt*2000+8000]+original_train_lst[cnt*2000+10000:]

def test_idx_list(cnt):
    original_train_lst=[i for i in range(16000)]
    return original_train_lst[cnt*2000:cnt*2000+2000]+original_train_lst[cnt*2000+8000:cnt*2000+10000]
    
if cnt>-1:
    train_idx=train_idx_list(cnt)
    test_idx=test_idx_list(cnt)

In [6]:
#문서 길이를 조정하는 코드 텍스트 리스트와 최대 길이를 입력하면 단어 시퀀스를 최대 길이 이내로 truncate 해줌
def limit_words(text_list, max_words):
    new_list = []
    for text in text_list:
        words = text.split()
        if len(words) > max_words:
            words = words[:max_words]
        new_text = ' '.join(words)
        new_list.append(new_text)
    return new_list

In [7]:
#사전학습된 버트는 최대 임베딩 토큰 수(512)가 정해져 있으므로 넉넉하게 최대길이를 300으로 제한
to_txt=limit_words(to_txt,300)

In [9]:
#불용어 불러오기
with open('C:/Users/user/Desktop/english.txt', 'r', encoding='utf-8') as file:
    stopwords = [line.strip() for line in file]

In [10]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words=stopwords)
X_dtm = vect.fit_transform(to_txt)
X_dtm = X_dtm.toarray()
X_new = SelectKBest(chi2, k=10000).fit(X_dtm, y)
TorF = X_new.get_support()
TorF
import numpy as np
word_view=np.array(vect.get_feature_names())
sw=word_view[TorF]

C:\Users\user\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [11]:
#선별된 10000개 단어
list(sw)

['aag',
 'aames',
 'aardman',
 'ab',
 'abbot',
 'abc',
 'abducted',
 'abe',
 'abhay',
 'abhishek',
 'abigails',
 'abject',
 'abominable',
 'abomination',
 'abominations',
 'aborigine',
 'aborigines',
 'abortion',
 'abovebr',
 'absolute',
 'absolutely',
 'absorbing',
 'absurd',
 'absurdbr',
 'absurdly',
 'abu',
 'abuses',
 'abysmal',
 'abysmally',
 'academy',
 'accent',
 'accentbr',
 'accents',
 'accentsbr',
 'accentuated',
 'accept',
 'acceptable',
 'acceptance',
 'accessible',
 'acclaimed',
 'accolade',
 'accommodate',
 'accomplices',
 'accomplished',
 'accountant',
 'accurate',
 'accused',
 'accustomed',
 'acharya',
 'achieved',
 'achievement',
 'achievements',
 'achieves',
 'achilleas',
 'acid',
 'acidic',
 'ackland',
 'acquire',
 'acquired',
 'acs',
 'acting',
 'actingbr',
 'action',
 'actionpacked',
 'active',
 'activism',
 'activist',
 'activists',
 'actors',
 'actorsactresses',
 'actual',
 'ada',
 'adamson',
 'adaptationbr',
 'adapted',
 'adapts',
 'add',
 'addicted',
 'addictio

In [12]:
feature_lst10000=sw

## 2) textcuboid 생성

In [13]:
x_train=to_txt[:16000]
x_test=to_txt[16000:]

In [14]:
doc_lst=[]
for sen in x_train:
    doc_lst.append(sen.split())

In [15]:
test_lst=[]
for sen in x_test:
    test_lst.append(sen.split())

In [16]:
#Train_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(16000):
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Train_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Train_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

Train_data에서 가장 많은 feature를 가진 문서의 경우 feature 102  개를 가짐
Train_data에서 가장 적은 feature를 가진 문서의 경우 feature 1  개를 가짐


In [17]:
#Test_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(4000):
    
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Test_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Test_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

Test_data에서 가장 많은 feature를 가진 문서의 경우 feature 83  개를 가짐
Test_data에서 가장 적은 feature를 가진 문서의 경우 feature 2  개를 가짐


In [18]:
#1-Channel TextCuboid 생성
textcuboid=[]

if cnt>-1:  #cross vaidation을 위한 2번째 이후의 시행인 경우
    for i in train_idx:
        frame1=np.zeros((102,768))  #(maximum number of features, 768)
        idx_cnt=0
        for j in range(10000):
            if feature_lst10000[j] in doc_lst[i]:
                #문서에서 선별한 단어(feature)의 위치를 찾아 임베딩 벡터 추출
                frame1[idx_cnt]=np.load('./bert_embedding/train(bert)/doc%d.npy'%i)[doc_lst[i].index(feature_lst10000[j])]
                idx_cnt+=1
        textcuboid.append(frame1)   #train data의 일부 문서에 대한 TexrCuboid가 추가됨
        
    for i in range(4000):
        frame1=np.zeros((102,768))   #(maximum number of features, 768)
        idx_cnt=0
        for j in range(10000):
            if feature_lst10000[j] in test_lst[i]:
                #문서에서 선별한 단어(feature)의 위치를 찾아 임베딩 벡터 추출
                frame1[idx_cnt]=np.load('./bert_embedding/test(bert)/test%d.npy'%i)[test_lst[i].index(feature_lst10000[j])]
                idx_cnt+=1
        textcuboid.append(frame1)   #test data의 문서에 대한 TexrCuboid가 추가됨
        

    textcuboid=np.array(textcuboid)

    np.save('./1-Channel textcuboid_IMDB(bert).npy',textcuboid)   
    
else:   #첫번째 시행인 경우
    for i in range(16000):
        frame1=np.zeros((102,768))   #(maximum number of features, 768)
        idx_cnt=0
        for j in range(10000):
            if feature_lst10000[j] in doc_lst[i]:
                #문서에서 선별한 단어(feature)의 위치를 찾아 임베딩 벡터 추출
                frame1[idx_cnt]=np.load('./bert_embedding/train(bert)/doc%d.npy'%i)[doc_lst[i].index(feature_lst10000[j])]
        textcuboid.append(frame1)

    textcuboid=np.array(textcuboid)

    np.save('./1-Channel textcuboid_IMDB(bert).npy',textcuboid)

In [19]:
#1-Channel TextCuboid 생성
textcuboid_test=[]

if cnt>-1:
    for i in test_idx:
        frame1=np.zeros((102,768))   #(maximum number of features, 768)
        idx_cnt=0
        for j in range(10000):
            if feature_lst10000[j] in doc_lst[i]:
                #문서에서 선별한 단어(feature)의 위치를 찾아 임베딩 벡터 추출
                frame1[idx_cnt]=np.load('./bert_embedding/train(bert)/doc%d.npy'%i)[doc_lst[i].index(feature_lst10000[j])]
                idx_cnt+=1
        textcuboid_test.append(frame1)     #train data의 일부 문서에 대한 TexrCuboid가 추가됨
        
    textcuboid_test=np.array(textcuboid_test)
    np.save('./1-Channel textcuboid_test_IMDB(bert).npy',textcuboid_test)
    
else:
    for i in range(4000):
        frame1=np.zeros((102,768))   #(maximum number of features, 768)
        idx_cnt=0
        for j in range(10000):
            if feature_lst10000[j] in test_lst[i]:
                #문서에서 선별한 단어(feature)의 위치를 찾아 임베딩 벡터 추출
                frame1[idx_cnt]=np.load('./bert_embedding/test(bert)/test%d.npy'%i)[test_lst[i].index(feature_lst10000[j])]
                idx_cnt+=1
        textcuboid_test.append(frame1)

    textcuboid_test=np.array(textcuboid_test)
    np.save('./1-Channel textcuboid_test_IMDB(bert).npy',textcuboid_test)